# Redis LangChain OpenAI eCommerce Chatbot

In [4]:
# pip install openai==0.28.1
!pip install langchain==0.0.336
# Install requirements
# !pip install open
# ! pip install -r requirements.txt
!pip install -U langchain-community

  Using cached langchain-0.0.336-py3-none-any.whl.metadata (16 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
Using cached langchain-0.0.336-py3-none-any.whl (2.0 MB)
Using cached langsmith-0.0.92-py3-none-any.whl (56 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.60
    Uninstalling langsmith-0.1.60:
      Successfully uninstalled langsmith-0.1.60
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.0
    Uninstalling langchain-0.2.0:
      Successfully uninstalled langchain-0.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.2.1 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.
langchain-community 0.2.0 requires langchain<0.3.0,>=0.2.0, but you have langchain 0.0.336 which is incompatible.
langchain-community 0.2.0 

In [5]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Download the dataset
# !gdown --id 1tHWB6u3yQCuAgOYc-DxtZ8Mru3uV5_lj

## Preprocess dataset

In [1]:
import pandas as pd

MAX_TEXT_LENGTH=512

def auto_truncate(val):
    """Truncate the given text."""
    return val[:MAX_TEXT_LENGTH]

# Load Product data and truncate long text fields
all_prods_df = pd.read_csv("product_data.csv", converters={
    'bullet_point': auto_truncate,
    'item_keywords': auto_truncate,
    'item_name': auto_truncate
})

In [2]:
# Contruct a primary key from item ID and domain name
all_prods_df['primary_key'] = (
    all_prods_df['item_id'] + '-' + all_prods_df['domain_name']
)
# Replace empty strings with None and drop
all_prods_df['item_keywords'].replace('', None, inplace=True)
all_prods_df.dropna(inplace=True)
# all_prods_df

# Reset pandas dataframe index
all_prods_df.reset_index(drop=True, inplace=True)

all_prods_df.head()

/tmp/ipykernel_696/1781557686.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_prods_df['item_keywords'].replace('', None, inplace=True)


,item_id,marketplace,country,main_image_id,domain_name,bullet_point,item_keywords,material,brand,color,item_name,model_name,model_number,product_type,primary_key
0,B07T2JY31Y,Amazon,IN,71vX7qIEAIL,amazon.in,3D Printed Hard Back Case Mobile Cover for Son...,mobile cover back cover mobile case phone case...,Wood,Amazon Brand - Solimo,others,Amazon Brand - Solimo Designer Leaf on Wood 3D...,Sony Xperia Z1 L39H,gz8056-SL40528,CELLULAR_PHONE_CASE,B07T2JY31Y-amazon.in
1,B0854774X5,Amazon,IN,81xaJCVnl3L,amazon.in,"Snug fit for Nokia 8.1, with perfect cut-outs ...",Back Cover Designer Case Designer Take It Easy...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Take It Easy UV...,Nokia 8.1,UV10714-SL40617,CELLULAR_PHONE_CASE,B0854774X5-amazon.in
2,B085494J1X,Amazon,IN,71iB+TUHVyL,amazon.in,"Snug fit for Lenovo Z6 Pro, with perfect cut-o...",Back Cover Designer Case Designer Panda Textur...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Panda Texture U...,Lenovo Z6 Pro,UV10833-SL40408,CELLULAR_PHONE_CASE,B085494J1X-amazon.in
3,B08511FPLP,Amazon,IN,81Gd4qnyogL,amazon.in,"Snug fit for Samsung Galaxy M31, with perfect ...",Samsung Galaxy M31 mobile case Samsung Galaxy ...,Plastic,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Old Stambh 3D P...,Samsung Galaxy M31,gz8759-SL40523,CELLULAR_PHONE_CASE,B08511FPLP-amazon.in
4,B08542LQNW,Amazon,IN,719JbIZ9gqL,amazon.in,"Snug fit for Huawei Y9 (2019), with perfect cu...",Back Cover Designer Case Designer Happy Mobile...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Happy UV Printe...,Huawei Y9 (2019),UV10721-SL13243,CELLULAR_PHONE_CASE,B08542LQNW-amazon.in


In [3]:
# Num products to use (subset)
NUMBER_PRODUCTS = 2500  

# Get the first 1000 products with non-empty item keywords
product_metadata = ( 
    all_prods_df
     .head(NUMBER_PRODUCTS)
     .to_dict(orient='index')
)

In [4]:
# Check one of the products
product_metadata[0]

{'item_id': 'B07T2JY31Y',
 'marketplace': 'Amazon',
 'country': 'IN',
 'main_image_id': '71vX7qIEAIL',
 'domain_name': 'amazon.in',
 'bullet_point': '3D Printed Hard Back Case Mobile Cover for Sony Xperia Z1 L39H Easy to put & take off with perfect cutouts for volume buttons, audio & charging ports. Stylish design and appearance, express your unique personality. Extreme precision design allows easy access to all buttons and ports while featuring raised bezel to life screen and camera off flat surface. Slim Hard Back Cover No Warranty',
 'item_keywords': 'mobile cover back cover mobile case phone case mobile panel phone panel LG mobile case LG phone cover LG back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Sony Xperia mobile case Sony Xperia phone cover Sony Xperia back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Sony Xperia mobile case Sony Xperia phone 

## Set up Redis as a vector db

In [5]:
# from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.redis import Redis as RedisVectorStore

# data that will be embedded and converted to vectors
texts = [
    v['item_name'] for k, v in product_metadata.items()
]

# product metadata that we'll store along our vectors
metadatas = list(product_metadata.values())

# we will use OpenAI as our embeddings provider
embedding = OpenAIEmbeddings(openai_api_key="open-key", model="text-embedding-3-small")

# name of the Redis search index to create
index_name = "products"

# assumes you have a redis stack server running on within your docker compose network
redis_url = "redis://redis:6379"

# create and load redis with documents
vectorstore = RedisVectorStore.from_texts(
    texts=texts,
    metadatas=metadatas,
    embedding=embedding,
    index_name=index_name,
    redis_url=redis_url
)

## Build the ChatBot with ConversationalRetrieverChain

In [6]:
# from langchain.callbacks.base import CallbackManager
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import (
    ConversationalRetrievalChain,
    LLMChain
)
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts.prompt import PromptTemplate

template = """Given the following chat history and a follow up question, rephrase the follow up input question to be a standalone question.
Or end the conversation if it seems like it's done.

Chat History:\"""
{chat_history}
\"""

Follow Up Input: \"""
{question}
\"""

Standalone question:"""

condense_question_prompt = PromptTemplate.from_template(template)

template = """You are a friendly, conversational retail shopping assistant. Use the following context including product names, descriptions, and keywords to show the shopper whats available, help find what they want, and answer any questions.
It's ok if you don't know the answer.

Context:\"""
{context}
\"""

Question:\"
\"""

Helpful Answer:"""

qa_prompt= PromptTemplate.from_template(template)


# define two LLM models from OpenAI
llm = OpenAI(temperature=0)

streaming_llm = OpenAI(
    streaming=True,
    callback_manager=CallbackManager([
        StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature=0.2,
    max_tokens=150
)

# use the LLM Chain to create a question creation chain
question_generator = LLMChain(
    llm=llm,
    prompt=condense_question_prompt
)

# use the streaming LLM to create a question answering chain
doc_chain = load_qa_chain(
    llm=streaming_llm,
    chain_type="stuff",
    prompt=qa_prompt
)


chatbot = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(),
    combine_docs_chain=doc_chain,
    question_generator=question_generator
)

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ConversationalRetrievalChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use create_history_aware_retriever together with create_retrieval_chain (see example in docstring) instead.

In [ ]:
# create a chat history buffer
chat_history = []

# gather user input for the first question to kick off the bot
question = input("Hi! What are you looking for today?")

# keep the bot running in a loop to simulate a conversation
while True:
    result = chatbot(
        {"question": question, "chat_history": chat_history}
    )
    print("\n")
    chat_history.append((result["question"], result["answer"]))
    question = input()

Hi! What are you looking for today? fancy footwear for going out


/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
Metadata key item_id not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bullet_point', 'item_keywords', 'material', 'brand', 'color', 'item_name', 'model_name', 'model_number', 'product_type', 'primary_key']
Metadata key marketplace not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bullet_point', 'item_keywords', 'material', 'brand', 'color', 'item_name',

 Hello there! Welcome to our store. Are you looking for some stylish and comfortable footwear? We have a great selection of women's slippers, sandals, and sneakers. We have the Teen Mix by Catwalk pink fashion slippers in a size 6 UK, the DARLING-1W3-004 open toe sandals in gold, and the Concept 3 by Skechers slip-on sneakers in navy and silver, all available in various sizes. We also have the RED WAGON girls' suede sandals in silver, perfect for your little one in a size 12 UK child. Is there a specific style or size you are looking for? I would be happy to help you find the perfect fit.



 These are nice. However, I am looking for men's shoes. Can you help me?


score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
Metadata key item_id not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bullet_point', 'item_keywords', 'material', 'brand', 'color', 'item_name', 'model_name', 'model_number', 'product_type', 'primary_key']
Metadata key marketplace not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bullet_point', 'item_keywords', 'material', 'brand', 'color', 'item_name', 'model_name', 'model_number', 'product_type', 'primary_key']
Metadata key country not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bu

...

score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
Metadata key item_id not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bullet_point', 'item_keywords', 'material', 'brand', 'color', 'item_name', 'model_name', 'model_number', 'product_type', 'primary_key']
Metadata key marketplace not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bullet_point', 'item_keywords', 'material', 'brand', 'color', 'item_name', 'model_name', 'model_number', 'product_type', 'primary_key']
Metadata key country not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bu

 Hello! Welcome to our store. We have a variety of formal shoes available, including the Amazon Brand - Arthur Harvey Men's Leather Formal Shoes and the Amazon Brand - Symbol Men's Formal Shoes. Both of these options are high-quality and stylish choices for any formal occasion. Is there a specific size or color you are looking for? I would be happy to help you find the perfect pair.



 could you provid me regions of stores?


score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
Metadata key item_id not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bullet_point', 'item_keywords', 'material', 'brand', 'color', 'item_name', 'model_name', 'model_number', 'product_type', 'primary_key']
Metadata key marketplace not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bullet_point', 'item_keywords', 'material', 'brand', 'color', 'item_name', 'model_name', 'model_number', 'product_type', 'primary_key']
Metadata key country not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bu

 Hello! Welcome to our store. Are you looking for storage furniture or a tote bag? We have the Iris Ohyama Storage Furniture with 4 compartments, perfect for organizing your belongings. We also have the Drop Avalon Shopper Tote Bag, which is great for carrying your essentials. Is there anything specific you are looking for? I'm happy to help you find what you need. We also have some essential oils, like the Sweet Orange from Whole Foods Market. Would you like to try it out? Let me know if you have any questions.



## Customize your chains for even better performance

In [17]:
import json

from langchain.schema import BaseRetriever
from langchain.vectorstores import VectorStore
from langchain.schema import Document
from pydantic import BaseModel


class RedisProductRetriever(BaseRetriever, BaseModel):
    vectorstore: VectorStore

    class Config:
        
        arbitrary_types_allowed = True

    def combine_metadata(self, doc) -> str:
        metadata = doc.metadata
        return (
            "Item Name: " + metadata["item_name"] + ". " +
            "Item Description: " + metadata["bullet_point"] + ". " +
            "Item Keywords: " + metadata["item_keywords"] + "."
        )

    def get_relevant_documents(self, query):
        docs = []
        for doc in self.vectorstore.similarity_search(query):
            content = self.combine_metadata(doc)
            docs.append(Document(
                page_content=content,
                metadata=doc.metadata
            ))
        return docs

### Setup ChatBot with new retriever

In [9]:
redis_product_retriever = RedisProductRetriever(vectorstore=vectorstore)

chatbot = ConversationalRetrievalChain(
    retriever=redis_product_retriever,
    combine_docs_chain=doc_chain,
    question_generator=question_generator
)

### Retry

In [ ]:
# create a chat history buffer
chat_history = []

# gather user input for the first question to kick off the bot
question = input("Hi! What are you looking for today?")

# keep the bot running in a loop to simulate a conversation
while True:
    result = chatbot(
        {"question": question, "chat_history": chat_history}
    )
    print("\n")
    chat_history.append((result["question"], result["answer"]))
    question = input()

Hi! What are you looking for today? fancy footwear for going out


score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
Metadata key item_id not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bullet_point', 'item_keywords', 'material', 'brand', 'color', 'item_name', 'model_name', 'model_number', 'product_type', 'primary_key']
Metadata key marketplace not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bullet_point', 'item_keywords', 'material', 'brand', 'color', 'item_name', 'model_name', 'model_number', 'product_type', 'primary_key']
Metadata key country not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bu

 Hello! Welcome to our store. Are you looking for some stylish and comfortable footwear? We have a variety of options for you to choose from. We have the Teen Mix By Catwalk Women's Pink Fashion Slippers, DARLING-1W3-004 Women’s Open Toe Sandals, Concept 3 by Skechers Easy Street Mesh Slip-on Sneaker, and RED WAGON Girls’ Suede Sandals. Is there a specific style or size you are interested in? I would be happy to help you find the perfect fit.



 These are nice. However, I am looking for men's shoes. Can you help me?


score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
Metadata key item_id not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bullet_point', 'item_keywords', 'material', 'brand', 'color', 'item_name', 'model_name', 'model_number', 'product_type', 'primary_key']
Metadata key marketplace not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bullet_point', 'item_keywords', 'material', 'brand', 'color', 'item_name', 'model_name', 'model_number', 'product_type', 'primary_key']
Metadata key country not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name', 'bu

 Hello! Welcome to our store. Are you looking for men's shoes? We have a variety of options available. We have the Men's Atwood Chelsea Boots in black, size US 10. We also have the Amazon Brand - Symbol Men's Casual Shoes. Are you interested in a different style? We also have the find. Men's Um1527 Chukka Boots in brown and tan, size US 5. And we have the find. Mens Albie Chukka Boots as well. Is there a specific style or size you are looking for? I'd be happy to help you find what you need.



In [ ]:
could you provid me regions of stores?

